In [1]:
import pandas as pd
import math
import numpy as np
import random

In [2]:
data = pd.read_excel("data.xlsx")
data

,Latitude,Longitude,Demand
0,4.3555,113.9777,5
1,4.3976,114.0049,8
2,4.3163,114.0764,3
3,4.3184,113.9932,6
4,4.4024,113.9896,5
5,4.4142,114.0127,8
6,4.4804,114.0734,3
7,4.3818,114.2034,6
8,4.4935,114.1828,5
9,4.4932,114.1322,8


# Define function

In [3]:
def cal_distance(point1, point2):
    """
    point1 and point2 = (latitude, longitude)
    return distance in km
    """
    return 100 * math.sqrt(((point2[1] - point1[1])**2) + ((point2[0] - point1[0])**2))

In [4]:
# initialize a n x n matrix of 0
def matrix(num_customer, dtype):
    n = num_customer + 1
    matrix = np.zeros((n, n), dtype=dtype)
    
    return matrix

In [5]:
def random_location(unvisited, cur_location=0, from_depot=True, vehicle=None):
    """
    randomly select next location
    
    from_depot = True if going from depot, false otherwise
    vehicle = "A" or "B" if not going from depot, None otherwise
    """
    
    # if going from depot
    if from_depot:
        temp_1 = pher_a[cur_location][1:] * nu_a[cur_location][1:]**beta
        temp_1 = temp_1 * unvisited
        temp_2 = pher_b[cur_location][1:] * nu_b[cur_location][1:]**beta
        temp_2 = temp_2 * unvisited
        
        temp = np.hstack((temp_1, temp_2))
        prob = temp / np.sum(temp)
         
        return np.random.choice(location_both, p=prob)
    
    # if going from other location 
    else:
        if vehicle == "A":
            temp = pher_a[cur_location][1:] * nu_a[cur_location][1:]**beta
            temp = temp * unvisited
            prob = temp / np.sum(temp)

            return np.random.choice(location, p=prob)
        
        elif vehicle == "B":
            temp = pher_b[cur_location][1:] * nu_b[cur_location][1:]**beta
            temp = temp * unvisited
            prob = temp / np.sum(temp)
            
            return np.random.choice(location, p=prob)
        

In [6]:
def local_update(cur_location, to_visit, vehicle, pher_a, pher_b):
    """
    perform local pheromone update
    
    first 2 parameters are the location in integer
    vehicle = "A" or "B"
    """
    if vehicle == "A":
        pher_a[cur_location][to_visit] = (1-alpha)*pher_a[cur_location][to_visit] + alpha*tau_zero
    elif vehicle == "B":
        pher_b[cur_location][to_visit] = (1-alpha)*pher_b[cur_location][to_visit] + alpha*tau_zero
   
    return pher_a, pher_b

In [7]:
def construct_solution(num_customer, data, pher_a, pher_b):
    """
    construct one feasible solution
    
    return :
    route_a and route_b indicating routes taken by vehicle a and b 
    route_list contains all routes in the solution  
    
    """
    unvisited = [1 for i in range(num_customer)]
    
    route_a = matrix(num_customer, np.uint8)
    route_b = matrix(num_customer, np.uint8)
    route_list = []
    cur_route = []
    
    # loop while there is unvisited location
    while any(unvisited):  
        
        # if going from depot
        if len(cur_route) == 0:
            # to_visit is the location (starting from 1), not the index
            to_visit = random_location(unvisited)
            
            # taking vehicle B
            if to_visit > num_customer:
                to_visit = to_visit - num_customer
                
                route_b[0][to_visit] = 1
                
                cur_route.append("B")
                cur_route.append(0)
                cur_route.append(to_visit)
                
                unvisited[to_visit-1] = 0
            
                capacity = 30 - data.loc[to_visit-1]["Demand"]
                
                # local update pheromone
                pher_a, pher_b = local_update(0, to_visit, "B", pher_a, pher_b)
            
            # taking vehicle A
            else:
                route_a[0][to_visit] = 1

                cur_route.append("A")
                cur_route.append(0)
                cur_route.append(to_visit)

                unvisited[to_visit-1] = 0

                capacity = 25 - data.loc[to_visit-1]["Demand"]
                
                # local update pheromone
                pher_a, pher_b =local_update(0, to_visit, "A", pher_a, pher_b)
                
        # not going from depot
        else:
            # check capacity
            demand = data["Demand"]
            available = unvisited * np.where(demand <= capacity, 1, 0)
            
            # if enough capcity for the remaining location
            if any(available):
                cur_location = cur_route[-1]
                
                # random select from available location
                to_visit = random_location(available, cur_location=cur_location, from_depot=False, vehicle=cur_route[0])
                
                if cur_route[0] == "A":
                    route_a[cur_route[-1]][to_visit] = 1
                else:
                    route_b[cur_route[-1]][to_visit] = 1
                    
                # local update pheromone
                pher_a, pher_b = local_update(cur_route[-1], to_visit, cur_route[0], pher_a, pher_b)
                
                # append location to route
                cur_route.append(to_visit)

                # update capacity
                capacity = capacity - demand[to_visit-1]

                unvisited[to_visit-1] = 0
            
            # if no capacity, go back to depot
            else:
                if cur_route[0] == "A":
                    route_a[cur_route[-1]][0] = 1
                else:
                    route_b[cur_route[-1]][0] = 1
                
                # local update pheromone
                pher_a, pher_b = local_update(cur_route[-1], 0, cur_route[0], pher_a, pher_b)
                cur_route.append(0)
                route_list.append(cur_route)
                cur_route = []
        
        # check if still has unvisited
        if not any(unvisited):
            if cur_route[0] == "A":
                route_a[cur_route[-1]][0] = 1
            else:
                route_b[cur_route[-1]][0] = 1
            
            # local update pheromone
            pher_a, pher_b = local_update(cur_route[-1], 0, cur_route[0], pher_a, pher_b)
            cur_route.append(0)
            route_list.append(cur_route)
            cur_route = []
    
    return route_a, route_b, route_list, pher_a, pher_b

In [8]:
def global_update(best_matrix_a, best_matrix_b, pher_a, pher_b):
    """
    parameters are the route matrix of globally best tour 
    perform global pheromones update
    """
    pher_a = ((1-alpha)*pher_a) + (alpha* best_matrix_a * nu_a)
    pher_b = ((1-alpha)*pher_b) + (alpha* best_matrix_b * nu_b)
    
    return pher_a, pher_b

In [9]:
def cal_cost(matrix_a, matrix_b):
    """
    calculate the total cost 
    """
    return np.sum(matrix_a * cost_a) + np.sum(matrix_b * cost_b)
    

### Declare variables to be used later

In [10]:
num_customer = len(data)
depot = (4.4184, 114.0932)

# used in random_location function
location = [i for i in range(1,num_customer+1)]
location_both = [i for i in range(1, 2*num_customer+1)]

distance_matrix = matrix(num_customer, dtype=np.float32)

# calculate distance between depot and locations
for i in range(num_customer):
    point2 = (data.loc[i]["Latitude"], data.loc[i]["Longitude"])
    distance = cal_distance(depot, point2)
    distance_matrix[0][i+1] = distance
    distance_matrix[i+1][0] = distance

# calculate distance betwee locations 
# loop through dataframe
for i in range(num_customer):
    point1 = (data.loc[i]["Latitude"], data.loc[i]["Longitude"])
    for j in range(i+1, num_customer):
        point2 = (data.loc[j]["Latitude"], data.loc[j]["Longitude"])
        distance = cal_distance(point1, point2)
        distance_matrix[i+1][j+1] = distance
        distance_matrix[j+1][i+1] = distance
        
cost_a = distance_matrix.copy()
cost_b = distance_matrix.copy()

# calculate the cost based on distance
cost_a = cost_a * 1.2
cost_b = cost_b * 1.5


### Parameter

In [11]:
beta = 2  # importance of pheromones

tau_zero = 0.5  # initial pheromones

alpha = 0.1   # pheromone decay/ evaporation rate 

iteration = 1000   # number of iterations

m = 10  # number of ants

In [12]:
# initialize pheromone
pher_a = matrix(num_customer, np.float32) 
pher_b = matrix(num_customer, np.float32) 

pher_a[pher_a==0] = tau_zero 
pher_b[pher_b==0] = tau_zero

nu_a = np.where(cost_a == 0, 0, 1/cost_a)
nu_b = np.where(cost_b == 0, 0, 1/cost_b)

C:\Users\Hp\AppData\Local\Temp\ipykernel_24140\3319566675.py:8: RuntimeWarning: divide by zero encountered in true_divide
  nu_a = np.where(cost_a == 0, 0, 1/cost_a)
C:\Users\Hp\AppData\Local\Temp\ipykernel_24140\3319566675.py:9: RuntimeWarning: divide by zero encountered in true_divide
  nu_b = np.where(cost_b == 0, 0, 1/cost_b)


# Main flow

In [13]:
best_matrix_a = np.zeros((1,1))
best_matrix_b = np.zeros((1,1))
best_routes = []
best_cost = float("inf")

# loop iteration
for i in range(iteration):
    # loop each ant
    for j in range(m):
        # construct a solution, local update while constructing
        route_a, route_b, route_list, pher_a, pher_b = construct_solution(num_customer, data, pher_a, pher_b)
        
        # compare cost with best, swap if lower cost
        cur_cost = cal_cost(route_a, route_b)
        if cur_cost < best_cost:
            best_cost = cur_cost
            best_routes = route_list.copy()
            best_matrix_a = route_a.copy()
            best_matrix_b = route_b.copy()
    
    # perform global update
    pher_a, pher_b = global_update(best_matrix_a, best_matrix_b, pher_a, pher_b)
    
    if(i%100 == 0 or i == (iteration-1)):
        # print best cost for every 10 iteration
        print("Iteration %d : Best Cost : RM %.2f"%(i, best_cost))

Iteration 0 : Best Cost : RM 131.68
Iteration 100 : Best Cost : RM 125.96
Iteration 200 : Best Cost : RM 121.10
Iteration 300 : Best Cost : RM 121.10
Iteration 400 : Best Cost : RM 121.10
Iteration 500 : Best Cost : RM 121.10
Iteration 600 : Best Cost : RM 121.10
Iteration 700 : Best Cost : RM 121.10
Iteration 800 : Best Cost : RM 121.10
Iteration 900 : Best Cost : RM 121.10
Iteration 999 : Best Cost : RM 121.10


# Format output

In [14]:
def get_output(best_matrix_a, best_matrix_b, best_routes, distance_matrix):
    """
    to display the best output 
    """
    
    total_distance = np.sum(best_matrix_a * distance_matrix) + np.sum(best_matrix_b * distance_matrix)
    
    total_cost = np.sum(best_matrix_a * cost_a) + np.sum(best_matrix_b* cost_b)
    
    print("Total distance = %.3f km"%(total_distance))
    print("Total cost = RM %.2f\n\n"%(total_cost))
    
    demand = data["Demand"]
    
    # for each round trip
    for i in range(len(best_routes)):
        rt_demand = 0
        rt_distance = 0
        
        string = "Depot -> "
        vehicle_type = best_routes[i][0]
        print("Vehicle ", i+1, "(Type ", vehicle_type, "): ")
        
        for j in range(2, len(best_routes[i])-1):
            cur = best_routes[i][j]
            prev = best_routes[i][j-1]
            
            os_distance = distance_matrix[prev][cur]
            rt_distance += os_distance
            
            rt_demand += demand[cur-1]
            
            message = "C"+ str(cur)+ " ("+ "%.3f km) -> "%(os_distance)
            string += message
    
        # return to depot
        os_distance = distance_matrix[best_routes[i][-2]][0]
        rt_distance += os_distance
        
        string += "Depot (%.3f km)"%(os_distance)
        
        if vehicle_type =="A":
            rt_cost = rt_distance * 1.2
        else:
            rt_cost = rt_distance * 1.5
            
        print(string)
        print("Round Trip Distance: %.3f km, Cost: RM%.2f, Demand: %d\n"%(rt_distance, rt_cost, rt_demand))

In [15]:
get_output(best_matrix_a, best_matrix_b, best_routes, distance_matrix)

Total distance = 100.919 km
Total cost = RM 121.10


Vehicle  1 (Type  A ): 
Depot -> C3 (10.347 km) -> C4 (8.323 km) -> C1 (4.021 km) -> C5 (4.839 km) -> C7 (11.448 km) -> Depot (6.508 km)
Round Trip Distance: 45.486 km, Cost: RM54.58, Demand: 22

Vehicle  2 (Type  A ): 
Depot -> C10 (8.436 km) -> C9 (5.060 km) -> C8 (11.358 km) -> Depot (11.612 km)
Round Trip Distance: 36.466 km, Cost: RM43.76, Demand: 19

Vehicle  3 (Type  A ): 
Depot -> C6 (8.061 km) -> C2 (1.834 km) -> Depot (9.072 km)
Round Trip Distance: 18.967 km, Cost: RM22.76, Demand: 16



## Compare with nearest neighbour

In [16]:
# get the cost using nearest neighbour

unvisited = [1 for i in range(num_customer)]
    
route_a = matrix(num_customer, np.uint8)
route_b = matrix(num_customer, np.uint8)
route_list = []
cur_route = []

while any(unvisited):
    
    # from depot
    if len(cur_route) == 0:
        temp = cost_a[0][1:]
        temp = temp * unvisited
        mask = [temp == 0]
        to_visit = (np.ma.masked_array(temp, mask)).argmin() + 1
        
        route_a[0][to_visit] = 1
        cur_route.append("A")
        cur_route.append(0)
        cur_route.append(to_visit)

        unvisited[to_visit-1] = 0

        capacity = 25 - data.loc[to_visit-1]["Demand"]
    
    else:
        demand = data["Demand"]
        available = unvisited * np.where(demand <= capacity, 1, 0)
        # if enough capcity for the remaining location
        if any(available):
            cur_location = cur_route[-1]
            temp = cost_a[cur_location][1:]
            temp = temp * unvisited
            mask = [temp == 0]
            to_visit = (np.ma.masked_array(temp, mask)).argmin() + 1
            
            route_a[cur_route[-1]][to_visit] = 1
            # append location to route
            cur_route.append(to_visit)

            # update capacity
            capacity = capacity - demand[to_visit-1]

            unvisited[to_visit-1] = 0
        # if no capacity, go back to depot
        else:
            route_a[cur_route[-1]][0] = 1
            cur_route.append(0)
            route_list.append(cur_route)
            cur_route = []
        
    if not any(unvisited):

        route_a[cur_route[-1]][0] = 1
        cur_route.append(0)
        route_list.append(cur_route)
        cur_route = []
    
            
get_output(route_a, route_b, route_list, distance_matrix)

Total distance = 106.878 km
Total cost = RM 128.25


Vehicle  1 (Type  A ): 
Depot -> C7 (6.508 km) -> C10 (6.018 km) -> C9 (5.060 km) -> C8 (11.358 km) -> C3 (14.290 km) -> Depot (10.347 km)
Round Trip Distance: 53.582 km, Cost: RM64.30, Demand: 25

Vehicle  2 (Type  A ): 
Depot -> C6 (8.061 km) -> C2 (1.834 km) -> C5 (1.604 km) -> Depot (10.483 km)
Round Trip Distance: 21.981 km, Cost: RM26.38, Demand: 21

Vehicle  3 (Type  A ): 
Depot -> C1 (13.152 km) -> C4 (4.021 km) -> Depot (14.142 km)
Round Trip Distance: 31.315 km, Cost: RM37.58, Demand: 11

